# COMPROBACIÓN DE VALORES

En este notebook se comprueban los valores obtenidos tras intentar replicar la metodología de H. Wang *et al.*.

Importamos librerías:

In [1]:
# Permite ajustar la anchura de la parte útil de la libreta (reduce los márgenes)
from IPython.display import display, HTML
display(HTML("<style>.container{ width:98% }</style>"))

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import re
import os

%config InlineBackend.figure_format = "retina"

Importamos datasets necesarios y hacemos limieza. 

Posteriormente, creamos la función para replicar los resultados.

In [2]:
# datos general de todas las baterias
main=pd.read_excel('main.xlsx', sheet_name=-1)
main = main.dropna(subset=['Observed Score'])
# display(main)
# Directorio que contiene los archivos .xlsx
directorio = 'excel/'
archivos = os.listdir(directorio)
def severity_level(battery='500mAh1-20S0C'):
    '''
    Function to calculate the hazard severity level of a battery.
    Weighted coeffitiens are wA = 2 * cSclase, wB = 3 * cSclase, wC = 2 * cSclase,
    where cSclase = 95/6 and cOffset = 5 - cScale.
    '''
    battery=battery.replace('.xlsx', '')
    print(battery)
    # fila donde esta la info general
    battery_row = main.loc[main['FileName'] == battery]
    # capacidad y SOC de la bateria
    # print(battery_row['Capacity'])
    # print(battery_row['Capacity'].values)
    # if (battery == 'OE-10Ahr-NMC-40SOC-cell5' or battery=='OE-LFP10Ah-100SOC-Cell16-original') or battery=='OE-LFP10Ah-100SOC-Cell16':
    #     return None, None, None
    capacity, SOC = battery_row['Capacity'].values[0], battery_row['SOC'].values[0]

    # datos del experimento de la bateria
    for i in archivos:
        if battery.lower()+'.xlsx' == i.lower():
            ruta_archivo = 'excel/'+battery+'.xlsx'
            break
    df = pd.read_excel(ruta_archivo,sheet_name=-1)
    columnas_a_eliminar = df.filter(regex='Unnamed').columns
    df = df.drop(columns=columnas_a_eliminar)
    df = df.rename(columns={'Time (second)': 'TimeLoad (Sec)', 'Time (sec)': 'TimePenetr (Sec)','Time (sec) ': 'TimeTemp (sec)'})
    
    maxTemp = np.max(df['TC1 (°C)'])
    
    # weighted parameters
    cScale = 95/6
    cOffset = 5 - cScale
    wA, wB, wC = 2 * cScale, 3 * cScale, 2 * cScale
    wCap = capacity /10_000 
    wSOC = SOC / 100 
    
    # variables
    idMaxTemp = df['TC1 (°C)'].idxmax()
    idMinTempPrev = df['TC1 (°C)'].iloc[:idMaxTemp].idxmin()
    
    # temperature increase rate
    tempIncreaseRate=0
    for i in range(1,df.shape[0]):
        aux = (df['TC1 (°C)'].iloc[i] - df['TC1 (°C)'].iloc[i-1]) / (df['TimeTemp (sec)'].iloc[i] - df['TimeTemp (sec)'].iloc[i-1])
        if aux > tempIncreaseRate:
            tempIncreaseRate = aux
    
    # values used for calculation of voltage drop score 
    initialVoltage = df['Cell Voltage (V)'].iloc[0]
    voltageRange = df['Cell Voltage (V)'].max() - df['Cell Voltage (V)'].min()
    
    
    ######################################################################
    # indice del ultimo registro de la columna no nulo
    last_valid_index = df['Voltage (V)'].last_valid_index()
    # Accede al valor correspondiente en la columna
    last_valid_value = df.at[last_valid_index, 'Voltage (V)']
    finalVoltageChange =  initialVoltage - last_valid_value
    ######################################################################
    idPeakVoltage = df['Cell Voltage (V)'].idxmax()
    fila_siguiente2 = df[df['TimePenetr (Sec)'] >= df['TimePenetr (Sec)'].iloc[idPeakVoltage] + 2].index.min()
    voltageDrop2 = df['Cell Voltage (V)'].iloc[idPeakVoltage] -  df['Cell Voltage (V)'].iloc[fila_siguiente2]
    fila_siguiente5 = df[df['TimePenetr (Sec)'] >= df['TimePenetr (Sec)'].iloc[idPeakVoltage] + 5].index.min()
    voltageDrop5 = df['Cell Voltage (V)'].iloc[idPeakVoltage] -  df['Cell Voltage (V)'].iloc[fila_siguiente5]
    
    if maxTemp < 40:
        return battery, capacity, SOC, maxTemp, tempIncreaseRate, initialVoltage, voltageRange, finalVoltageChange, voltageDrop2,voltageDrop5, 1, 5  
    
    elif maxTemp > 160:
        return battery, capacity, SOC, maxTemp, tempIncreaseRate, initialVoltage, voltageRange, finalVoltageChange, voltageDrop2,voltageDrop5, 5, 100
    
    # voltage drop score
    # print('voltageRange/initialVoltage',voltageRange/initialVoltage)
    # print('finalVoltageChange/initialVoltage',finalVoltageChange/initialVoltage)
    # print('voltageDrop5/initialVoltage',voltageDrop5/initialVoltage)
    # print('voltageDrop2/initialVoltage',voltageDrop2/initialVoltage)
    if voltageRange/initialVoltage < 0.5:
        # a very small voltage change through the test
        # This item fits with situation that cell voltage has little drop through the test, so time is excluded in the calculation since it has insignificant impact on voltage change.
        voltageDropScore = 1 
    elif (voltageRange/initialVoltage > 0.7 and finalVoltageChange / initialVoltage > 0.7) and voltageDrop5/initialVoltage > 0.7:
        # fast voltage drop to an almost zero 
        # value in less than 5 seconds without any voltage recovery throughout the test
        voltageDropScore = 5
    elif voltageRange/initialVoltage > 0.5 and finalVoltageChange/initialVoltage < 0.2:
        # voltage drop is >50%, but it recovers to a value close to the initial voltage
        # cell voltage has significant change when the indenter applies load on cell, but voltage recovers close to initial voltage after test stops
        # energy release due to voltage drop is trivial in a short interval, so time effect is not considered in the calculation
        voltageDropScore = 2
    elif voltageDrop2/initialVoltage >= 0.4 and finalVoltageChange/initialVoltage > 0.7:
        # significant voltage drop is observed in 2 consecutive 
        # seconds, and the final measured voltage is also in a lower state 
        # even if a voltage recovery occurs shortly during the test.
        voltageDropScore = 4
    elif voltageDrop2/initialVoltage < 0.4 and finalVoltageChange/initialVoltage > 0.6:
        # e voltage drops to a relatively small amount in 2 consecutive seconds 
        #  and recovers to the initial voltage but drops again significantly to a 
        # lower value
        voltageDropScore = 3
    else:
        print('Not able to calculate voltage drop score :( ')
        return battery, capacity, SOC, maxTemp, tempIncreaseRate, initialVoltage, voltageRange, finalVoltageChange, voltageDrop2,voltageDrop5, None, None
    
    minimum = min([wA * (maxTemp/160)**0.25 + wB * (tempIncreaseRate / 200) + wC * wCap  * wSOC * voltageDropScore + cOffset, 100])
    
    return  battery, capacity, SOC, maxTemp, tempIncreaseRate, initialVoltage, voltageRange, finalVoltageChange, voltageDrop2,voltageDrop5, voltageDropScore, minimum 

C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


Preparamos el dataframe y generamos los datos para comprobar los resultados:

In [3]:
data = {
    'battery':[],
    'capacity':[], 
    'SOC':[], 
    'maxTemp':[], 
    'tempIncreaseRate':[],
    'initialVoltage':[], 
    'voltageRange':[], 
    'finalVoltageChange':[], 
    'voltageDrop2':[],
    'voltageDrop5':[],
    'voltageDropScore':[],
    'OHS':[],
    'CHS':[]
}
for i in range(len(main.FileName)):
    if ('Copy' not in main.FileName[i]) or ('copy' not in main.FileName[i]):
        valor = severity_level(main.FileName[i])
        data['battery']+=[valor[0]]
        data['capacity'] += [valor[1]]
        data['SOC'] += [valor[2]]
        data['maxTemp'] += [valor[3]]
        data['tempIncreaseRate'] += [valor[4]]
        data['initialVoltage'] += [valor[5]]
        data['voltageRange'] += [valor[6]]
        data['finalVoltageChange'] += [valor[7]]
        data['voltageDrop2'] += [valor[8]]
        data['voltageDrop5'] += [valor[9]]
        data['voltageDropScore'] += [valor[10]]
        if main['Observed Score.1'][i] == 1:
            data['OHS'] += [5]
        elif main['Observed Score.1'][i] == 2:
            data['OHS'] += [17.5]
        elif main['Observed Score.1'][i] == 3:
            data['OHS'] += [50]
        elif main['Observed Score.1'][i] == 4:
            data['OHS'] += [82.5]
        elif main['Observed Score.1'][i] == 5:
            data['OHS'] += [95]
        else:
            data['OHS'] += [100]
        data['CHS'] += [valor[11]]

df_data = pd.DataFrame(data)

Sorteria-100SOC-cell1
Sorteria-100SOC-cell2
Sorteria-Control-100SOC-cell1
Sorteria-Control-100SOC-cell2
LCO-LP-6400mAh-100SOC-cell1
LCO-LP-6400mAh-100SOC-cell2
LCO-LP-6400mAh-80SOC-Cell3
LCO-LP6400mAh-80SOC-cell4
Soetria-80SOC-cell3
Sorteria-Control-60SOC-cell1
Soetria-Control-45SOC-cell1
Sorteria-control-40SOC-cell2
Sorteria-control-20SOC-cell1
Soteria-control-30SOC-cell2
Soteria-control-30SOC-cell1
Soteria-control-10SOC-cell1
Soteria-Control-0SOC-cell1
Soteria-SCC-90SOC-cell1
Soteria-SCC-80SOC-cell1
Soteria-SCC-60SOC-cell1
Soteria-SCC-20SOC-5180mAh
OE-LCO-6470mAh-60SOC
OE-LCO-6490mAh-60SOC
OE-LCO-6270mAh-40SOC
OE-LCO-6500mAh-20SOC
OE-LCO-6560mAh-0SOC
OE-LCO-6550mAh-20SOC
OE-LCO-6500mAh-40SOC
OE-LCO-6340mAh-0SOC
OE-LCO-6400mAh-20SOC
Soteria-SCC-5070-70SOC
OE-LFP10Ah-0SOC-cell9
OE-LFP10Ah-0SOC-cell10
OE-LFP10Ah-40SOC-cell7
OE-LFP10Ah-40SOC-cell8
OE-LFP10Ah-70SOC-cell5
OE-LFP10Ah-70SOC-cell6
OE-LFP10Ah-80SOC-cell3
OE-LFP10Ah-80SOC-cell4
OE-10Ahr-NMC-70SOC-cell7
OE-10Ahr-NMC-80SOC-cell8


# COMPARACIÓN

Completamos el dataframe:

In [4]:
comp={
    'battery':[],
    'CHS':[],
    'myCHS':[]
}
for i in range(len(main)):
    for j in range(len(df_data)):
        if main.FileName[i] == df_data.battery[j]:
            comp['battery'] += [df_data.battery[j]]
            comp['CHS'] += [main['Calculated Score'][i]]
            comp['myCHS'] += [df_data['CHS'][j]]
df_comp = pd.DataFrame(comp)
df_comp

,battery,CHS,myCHS
0,Sorteria-100SOC-cell1,5.0000,5.00000
1,Sorteria-100SOC-cell2,5.0000,5.00000
2,Sorteria-Control-100SOC-cell1,100.0000,100.00000
3,Sorteria-Control-100SOC-cell2,100.0000,100.00000
4,LCO-LP-6400mAh-100SOC-cell1,100.0000,100.00000
...,...,...,...
113,Soteria-Control-5030mAh-10SOC,28.8759,32.29076
114,Soteria-SCC-4870mah-0SOC,5.0000,5.00000
115,Soteria-SCC-4630mAh-10SOC,5.0000,5.00000
116,Soteria-SCC-5080mAh-20SOC,5.0000,5.00000


### Valores coinciden:

In [5]:
df_comp[df_comp.CHS == df_comp.myCHS]

,battery,CHS,myCHS
0,Sorteria-100SOC-cell1,5.0,5.0
1,Sorteria-100SOC-cell2,5.0,5.0
2,Sorteria-Control-100SOC-cell1,100.0,100.0
3,Sorteria-Control-100SOC-cell2,100.0,100.0
4,LCO-LP-6400mAh-100SOC-cell1,100.0,100.0
5,LCO-LP-6400mAh-100SOC-cell2,100.0,100.0
17,Soteria-SCC-90SOC-cell1,5.0,5.0
18,Soteria-SCC-80SOC-cell1,5.0,5.0
19,Soteria-SCC-60SOC-cell1,5.0,5.0
20,Soteria-SCC-20SOC-5180mAh,5.0,5.0


In [ ]:
df_comp[df_comp.CHS == df_comp.myCHS].shape

(26, 3)

### Valores no coinciden:

In [ ]:
df_comp[(df_comp.CHS >= df_comp.myCHS+2.5) | (df_comp.CHS <= df_comp.myCHS-2.5)]

Intentamos dar un rango de error para considerar coincidentes:

In [ ]:
df_comp[(df_comp.CHS >= df_comp.myCHS+2.5) | (df_comp.CHS <= df_comp.myCHS-2.5)]

In [ ]:
df_comp[(df_comp.CHS >= df_comp.myCHS+2.5) | (df_comp.CHS <= df_comp.myCHS-2.5)].shape